# Produccion eléctrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a la produccion eléctrica de energias renovables y no renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto en principio es prácticamente inmediato, también vamos a centrarnos en la produccion de energia hidroelectrica para poder medir cuanto tiempo tarda en afectar a este tipo de energía las precipitaciones..

En base a esto tenemos varias cuestiones que queremos despejar:
- Que relación hay entre precipitaciones y el aumento de la generación de energia eléctrica hidráulica.
- Que relación hay entre horas de sol, temperatura y el aumento de la generación de energia eléctrica sólar.
- Que relación hay entre viento y el aumento de la generación de energia eléctrica eólica, esta relación esta condicionada por la temperatura, percipitaciones o horas de sol.
- Cuantos días tarda en aumentar la generación eléctrica de fuentes de energía renovables en función de los fenomenos metereológicos.
- Dados una prediccion meteorologica que valores de generación eléctrica tendremos para una fecha determinada.



Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Debido a la falta de datos a nivel diario de REE por provincia o comunidades autonomas, voy a centrar el analisis a nivel de sistema eléctrico (Peninsula, Baleares, Canarias ,Ceuta y Melilla).

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Requisitos para la ejecución del notebook:

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet) e instalar la libreria request.

Además las versiones de cada libreria utilizada en este proyecto son:

El modelo ha utilizar al tratarse de una prediccion númerica y no de obtener una etiqueta, sera una regresión.
Para poder llevar a cabo esa regresion se han pasado los valores de fechas a númericos, y se ha creado una columna por cada tipo de energia, creando varias variables objetivo.



In [1]:
# If the code is executed from Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/TFM_Generacion_electrica_AEMET/Python

Mounted at /content/drive
/content/drive/My Drive/TFM_Generacion_electrica_AEMET/Python


In [2]:
# Instalacion libreria AEMET para ingesta de datos
!pip install python-aemet

     |████████████████████████████████| 409kB 3.9MB/s 


In [3]:
from Lectura_AEMET_REE import Ingestion_AEMET,Ingestion_REE
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

Ing_AEMET=Ingestion_AEMET()

Ing_REE=Ingestion_REE()

pd.options.display.max_columns=None
plt.style.use('ggplot')



In [4]:
# put this in an input sentence
date_ini="2016-01-01T00:00:00UTC"
date_end="2021-12-31T00:00:00UTC"

# 1- Read data from API and CSV

###  Lectura datos AEMET <a id='From_api'></a>

Para la lectura de datos metereólogicos, vamos a utilizar la libreria aemet, de la que utilizaremos los metodos de las clases Aemet y Estacion para obtener los datos a nivel diario de cada estación meterologica para un rago de fechas, de las provincias que nos interesan para el estudio.<a id='From_api'></a>

In [5]:
df_weather=Ing_AEMET.read_weather_dates(date_ini,date_end)

Reading list of id of weather stations...



Reading AEMET data from 2016-01-01T00:00:00UTC to 2016-12-31T00:00:00UTC ...



Reading AEMET data from 2017-01-01T00:00:00UTC to 2017-12-31T00:00:00UTC ...



Reading AEMET data from 2018-01-01T00:00:00UTC to 2018-12-31T00:00:00UTC ...



Reading AEMET data from 2019-01-01T00:00:00UTC to 2019-12-31T00:00:00UTC ...



Reading AEMET data from 2020-01-01T00:00:00UTC to 2020-12-31T00:00:00UTC ...



Reading AEMET data from 2021-01-01T00:00:00UTC to 2021-12-31T00:00:00UTC ...



Finish reading AEMET date from 2016-01-01 to 2021-05-28


### Lectura de los datos de REE

Para la lectura de los datos de REE voy a utilizar la libreria python requests para a traves de su API, obtener la generación de electricidad en , para cada tipo de energia Electrica.

Una vez leidos los datos de la API, guardo los datos en formato json en un fichero. para no tener que repetir las consultas y poder trabajar sin conexion.

La estrutura de los datos leidos de REE es la siguiente:


Por lo que vamos a almacenar 2 ficheros:
    - Renovables
    - No Renovables



In [6]:
df_ree=Ing_REE.read_ree_dates(date_ini,date_end)

## Other data

https://datos.gob.es/en/catalogo/l01280796-calendario-laboral

In [7]:
df_holidays=pd.read_csv(Ing_AEMET.path_Data+'calendario.csv',
                        sep=';',
                        encoding = 'latin-1',
                        usecols=['Dia','Tipo de Festivo'],
                        dtype=str)
df_holidays['Holiday']=(df_holidays['Tipo de Festivo']=='Festivo nacional').astype(np.int)
df_holidays['Dia']=pd.to_datetime(df_holidays['Dia'])

df_holidays.drop(columns='Tipo de Festivo',inplace=True)
df_holidays.dropna(inplace=True)


In [8]:
df_weather['fecha']=pd.to_datetime(df_weather['fecha'])
df_weather=df_weather.merge(df_holidays,how='left',left_on='fecha',right_on='Dia')
df_weather['weekday']=df_weather['Dia'].dt.dayofweek
df_weather.drop(columns='Dia',inplace=True)


In [9]:
df_weather.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin,Holiday,weekday
0,2016-01-01,0252D,ARENYS DE MAR,BARCELONA,74,"11,2","0,2","7,8",06:20,"14,7",12:40,24,"1,7","7,2",14:00,NaN,NaN,NaN,NaN,NaN,1,4
1,2016-01-02,0252D,ARENYS DE MAR,BARCELONA,74,"11,8","0,0","7,5",23:40,"16,1",13:10,24,"1,1","9,7",11:40,NaN,NaN,NaN,NaN,NaN,0,5
2,2016-01-03,0252D,ARENYS DE MAR,BARCELONA,74,"10,0","0,0","5,8",05:50,"14,1",13:20,24,"1,9","9,2",16:00,NaN,NaN,NaN,NaN,NaN,0,6
3,2016-01-04,0252D,ARENYS DE MAR,BARCELONA,74,"11,6","0,9","8,0",22:50,"15,3",10:10,32,"0,8","8,9",02:40,NaN,NaN,NaN,NaN,NaN,0,0
4,2016-01-05,0252D,ARENYS DE MAR,BARCELONA,74,"9,9","0,1","5,6",23:59,"14,2",12:30,32,"2,5","8,6",17:30,NaN,NaN,NaN,NaN,NaN,1,1


#2- Saving Data to CSV

Para evitar tener que leer todos los datos en cada ejecucion, guardamos los en formato csv desde los DataFrame de REE y AEMET

In [10]:
Ing_AEMET.save_to_csv(df_weather)
Ing_REE.save_to_csv(df_ree) 